In [ ]:
from skimage.color import rgb2gray
from skimage import img_as_ubyte,img_as_float
import requests
from PIL import Image
from io import BytesIO
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

#ignore
###
###
###

image1_url = "https://www.usfca.edu/sites/default/files/styles/1_1_1536x1536/public/migrated/images/headshots/interian-yannet.jpg.jpeg?itok=96QLAl9n" 
image2_url = "https://www.usfca.edu/sites/default/files/styles/1_1_1536x1536/public/2023-01/victor_palacios%20%281%29.jpg.jpeg?itok=SgwZtGu4" 
image3_url = "https://www.usfca.edu/sites/default/files/styles/1_1_1536x1536/public/2022-07/Robert%20Clements.jpg.jpeg?itok=hn0BgUxA" 


response1 = requests.get(image1_url)
image1 = Image.open(BytesIO(response1.content))

response2 = requests.get(image2_url)
image2 = Image.open(BytesIO(response2.content))

response3 = requests.get(image3_url)
image3 = Image.open(BytesIO(response3.content))

from skimage import data
gray_images = {
        "cat":rgb2gray(img_as_float(data.chelsea())),
        "ex1":rgb2gray(img_as_float(data.astronaut())),
        "ex2":rgb2gray(img_as_float(data.stereo_motorcycle()[0])),
        "mystery1":rgb2gray(img_as_float(image1)),
        "mystery2":rgb2gray(img_as_float(image2)),
        "mystery3":rgb2gray(img_as_float(image3))
}

## Understanding SVD through Image Compression

SVD decomposes an $n$ x $p$ rectangular matrix $X$ into three factors:

$X = U D V^T$

 - $U$ = matrix of left singular vectors in the columns
 - $D$ = diagonal matrix with singular values
 - $V$ = matrix of right singular vectors in the columns

## Rank k approximations via SVD 

Recall that SVD can be written equivalently as:

$X = \sum_{j=1}^p \delta_j u_jv_j^T\\
    = \delta_1 u_1v_1^T + \delta_2 u_2v_2^T + ... + \delta_p u_pv_p^T$ 


SVD image compression will take advantage of the fact that past some point $k$, the singular values $\{\delta_j\}_{j>k}$ are very small. If we take their values as effectively $0$, we can reconstruct the images without needing to store all of the original values of the matrix. 

## skimage library


skimage is an image processing library (from the sci-kit family) for working with images in python. It has some built in images for us to work with. 

In [ ]:
gray_images["cat"]

In [ ]:
plt.imshow(gray_images["cat"], cmap='gray')
plt.show()

In [ ]:
gray_images["cat"].shape

## svd in python

The numpy.linalg library has an SVD function which returns $U,D,V_t$: 

- $U$ has left singular vectors in the columns
- $D$ is a rank 1 numpy array with singular values
- $V_t$ has right singular vectors in the rows - equivalent to $V^T$ in our notes

In [ ]:
from numpy.linalg import svd

Let's write a function which does SVD and approximates the image with a rank k reconstruction.

In [ ]:
def rank_k_approx(image,k):
    """
    Performs SVD decomposition. Truncates singular values at kth index & returns rank k approximation of image.
    
    --------
    Outputs: reconst_matrix, array of singular values s
    """
    U,D,Vt = svd(image, full_matrices=False)
    matrix_k = np.dot(U[:,:k],np.dot(np.diag(D[:k]),Vt[:k,:]))
       
    return matrix_k, D, Vt

Let's show the magnitude of the singular values on a plot, with a vertical line indicating where we truncated our approximation. We can also compare the reconstructed image with the truth to get an idea of how many rank 1 matrices we need to be "good enough" to see the cat.

In [ ]:
def show_images_gray(img_name, k, showtruth = False):
    """
    Compresses gray scale images to rank k and displays the approximate image.
    Plots singular values in a scree plot.
    """
    image=gray_images[img_name]
    original_shape = image.shape
    
    rankk_img, D, Vt = rank_k_approx(image, k)
    
    
    fig,axes = plt.subplots(1,3,figsize=(12,5))
    axes[0].plot(D)
    axes[0].axvline(k, color = "red")
    axes[0].set_title("Singular Value Scree Plot")
    
    #compression_ratio =100.0 * (k*(original_shape[0] + original_shape[1])+k)/(original_shape[0]*original_shape[1])
    axes[1].set_title("Rank {:.0f}".format(k)+" Reconstruction")
    axes[1].imshow(rankk_img,cmap='gray')
    axes[1].axis('off')
    
    if showtruth == True: 
        axes[2].set_title("True Image")
        axes[2].imshow(image,cmap='gray')
       # plt.show()
        axes[2].axis('off')
        
    fig.tight_layout()
    plt.show()

### Rank 1: not good enough

In [ ]:
show_images_gray("cat", k = 1, showtruth = False)

## Rank 5: better, but still hard to tell its really a cat

In [ ]:
show_images_gray("cat", k = 5, showtruth = False)

## Rank 10: I can kinda see it...

In [ ]:
show_images_gray("cat", k = 10, showtruth = False)

## Rank 20: Now it's pretty clear! 

In [ ]:
show_images_gray("cat", k = 20, showtruth = False)

In [ ]:
show_images_gray("cat", k = 50, showtruth = False)

## Let's try with some other mystery images before looking at the truth.

### Image 1

In [ ]:
show_images_gray("ex1", k = 1, showtruth = False)

In [ ]:
show_images_gray("ex1", k = 10, showtruth = False)

In [ ]:
show_images_gray("ex1", k = 25, showtruth = False)

...

### Image 2


In [ ]:
show_images_gray("ex2", k = 1, showtruth = False)

In [ ]:
show_images_gray("ex2", k = 5, showtruth = False)

In [ ]:
show_images_gray("ex2", k = 10, showtruth = False)

In [ ]:
show_images_gray("ex2", k = 25, showtruth = False)

## Mystery Img :o

In [ ]:
show_images_gray("mystery1", k = 1, showtruth = False)

In [ ]:
show_images_gray("mystery1", k = 5, showtruth = False)

In [ ]:
show_images_gray("mystery1", k = 10, showtruth = False)

In [ ]:
show_images_gray("mystery1", k = 50, showtruth = False)

In [ ]:
show_images_gray("mystery2", k = 1, showtruth = False)

In [ ]:
show_images_gray("mystery2", k = 3, showtruth = False)

In [ ]:
show_images_gray("mystery2", k = 5, showtruth = False)

In [ ]:
show_images_gray("mystery2", k = 10, showtruth = False)

In [ ]:
show_images_gray("mystery2", k = 50, showtruth = False)

In [ ]:
show_images_gray("mystery3", k = 1, showtruth = False)

In [ ]:
show_images_gray("mystery3", k = 3, showtruth = False)

In [ ]:
show_images_gray("mystery3", k = 5, showtruth = False)

In [ ]:
show_images_gray("mystery3", k = 10, showtruth = False)

In [ ]:
show_images_gray("mystery3", k = 50, showtruth = False)

### Color images

Color images are represented 3 dimensional numpy arrays - one array for each of the RGB values. We can apply SVD to each of the arrays (red, green, blue) separately and then stack them back together to reconstruct a color image.



In [ ]:
color_images = {
    "cat":img_as_float(data.chelsea()),
    "im1":img_as_float(data.astronaut()),
    "im2":img_as_float(data.stereo_motorcycle()[0]),
    "mystery1":rgb2gray(img_as_float(image1)),
    "mystery2":rgb2gray(img_as_float(image2)),
    "mystery3":rgb2gray(img_as_float(image3))
}

In [ ]:
def rank_k_color(img_name, k, showtruth = False):
    """
    Performs SVD on each of the RGB channels & restacks to make a color image.
    """
    image = color_images[img_name]
    original_shape = image.shape
    image_reconst_layers = [rank_k_approx(image[:,:,i],k)[0] for i in range(3)]
    image_reconst = np.zeros(image.shape)
    for i in range(3):
        image_reconst[:,:,i] = image_reconst_layers[i]
        
    fig,axes = plt.subplots(1,2,figsize=(12,5))

    
    #compression_ratio =100.0 * (k*(original_shape[0] + original_shape[1])+k)/(original_shape[0]*original_shape[1])
    axes[0].set_title("Rank {:.0f}".format(k)+" Reconstruction")
    axes[0].imshow(image_reconst,cmap='gray')
    axes[0].axis('off')
    
    if showtruth == True: 
        axes[1].set_title("True Image")
        axes[1].imshow(image,cmap='gray')
        axes[1].axis('off')
        plt.show()

In [ ]:
rank_k_color("cat", k = 10, showtruth = True)

In [ ]:
rank_k_color("cat", k = 20, showtruth = True)

Note: sometimes the values of a matrix are constrained to an interval, e.g. with RGB, the value must fall between [0, 1]. Sometimes our low rank approximation yields an entry which is outside of this range, which is what the warning is telling us. The default method for handling it is to "clip" the value. Ex: if the approximation was 1.02, we clip it to 1 so it falls in the valid range. 

In [ ]:
rank_k_color("im2", k = 3, showtruth = True)

In [ ]:
rank_k_color("im2", k = 20, showtruth = True)

# Lab Questions

## svd from "scratch"

(if we can use eigendecomp for free)

Write a function which performs SVD and has the option to return a rank k approximation, without using the built in svd function.

In [ ]:
from numpy.linalg import eig

...

## Simulation Ex:

Simulate a data matrix X by generating random values for the first few columns. Then add in extra columns which are nearly linearly dependent to the first few. Perform SVD and use a low rank approximation to reconstruct your original X matrix. How well does it do at recreating the data? Visualize the original and reconstructed data. 

### Exercises

1. How can we measure how closely a grayscale reconstruction approximates its true image? Define a reasonable error metric an write a function which calculates it given the rank k of the reconstruction.

2. The choice of k is somewhat subjective. For the following images, experiment with the choice of k and look at where it falls on the scree plot. Is there a similarity across images? What can the scree plot tell us about choosing k?

In [ ]:
gray_images = {
        "coffee":rgb2gray(img_as_float(data.coffee())),
        "camera":data.camera(),
        "coin": data.coins(),
        "clock":data.clock(),
        "blobs":data.binary_blobs(),
}

3. How can we measure how closely a color reconstruction approximates its true image? Define a reasonable error metric an write a function which calculates it given the rank k of the reconstruction.

4. Calculate the number of "parameters" or matrix entries it takes for a rank K SVD to approximate the original matrix. Write a function which takes the desired rank K, calculates this, and returns a compression ratio based off of it and the original number of entries in the image matrix.

5. Consider the following images. What do you notice about the rank needed in order to discern what the photo is of? Think about how resolution relates to rank, and how it changes if there is not a subject in an image. How does the notion of a subject relate to the statistical ideas of signal and noise?

In [ ]:
gray_images = {
        "grass":data.grass(),
        "gravel": data.gravel(),
        "brick":data.brick()}

In [ ]:
show_images_gray("grass", k = 10, showtruth = True)

In [ ]:
show_images_gray("gravel", k = 10, showtruth = False)

In [ ]:
show_images_gray("brick", k = 5, showtruth = False)